In [13]:
from ethpandaops_python.preprocessor import Preprocessor
from eip4844_blob_data.panel_charts import create_slot_inclusion_line_chart, create_priority_fee_chart, get_slot_inclusion_table
from eip4844_blob_data.polars_preprocess import create_slot_inclusion_df, create_slot_gas_bidding_df
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn
import time


nest_asyncio.apply()
pn.extension("tabulator", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [14]:
# labeled blobs - https://dune.com/queries/3521610
sequencers_l2: dict[list[str], list[str]] = {
    "sequencer_addresses": [
        # should be the "from" addresses, this is what hilldobby SQL query does.
        # should also be proper checksum, not lowercase
        "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc",
        "0x5050F69a9786F081509234F1a7F4684b5E5b76C9",
        "0x6887246668a3b87F54DeB3b94Ba47a6f63F32985",
        "0x000000633b68f5D8D3a86593ebB815b4663BCBe0",
        "0x415c8893D514F9BC5211d36eEDA4183226b84AA7",
        "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e",
        "0xcF2898225ED05Be911D3709d9417e86E0b4Cfc8f",
        "0x0D3250c3D5FAcb74Ac15834096397a3Ef790ec99",
        "0xC70ae19B5FeAA5c19f576e621d2bad9771864fe2",
        "0xC94C243f8fb37223F3EB2f7961F7072602A51B8B"
    ],
    "sequencer_names": [
        "arbitrum",
        "base",
        "optimism",
        "taiko",
        "blast",
        "linea",
        "scroll",
        "zksync",
        "paradex",
        "metal"
    ],
}

num_days: int = 7

In [15]:
def get_data() -> dict[str, pl.DataFrame]:
    return Preprocessor(
        blob_producer=sequencers_l2,
        period=num_days,
        network="mainnet",  # mainnet
    ).cached_data

In [16]:
def filter_data_seq(
    sequencers: list[str], slot_inclusion_joined_df: pl.DataFrame, cached_data: dict[str, pl.DataFrame]
) -> dict[str: pl.DataFrame]:
    """
    This function filters a dataframe and returns updated chart data, based on the input of the dashboard user.
    """

    # slot inclusion
    slot_inclusion_df = (
        slot_inclusion_joined_df.filter(
            pl.col("sequencer_names").is_in(sequencers))
        .filter(pl.col('meta_network_name') == 'mainnet')
        .unique()
        .sort(by="slot")
    )

    # gas bidding scatterplot median
    slot_gas_groupby_df = (
        slot_inclusion_df.group_by("slot_inclusion_rate", "sequencer_names")
        .agg(
            pl.col("priority_fee_bid_percent_premium").median(),
            pl.col("base_fee_per_gas").mean(),
        )
        .sort(by="slot_inclusion_rate")
        .drop_nulls()
    )

    return {
        # time series
        "slot_inclusion_df": slot_inclusion_df,
        "slot_gas_groupby_df": slot_gas_groupby_df,
    }

In [17]:
# start dashboard
def start_interactive_panel(filtered_data_dict, sequencer_names_list):
    multi_select = pn.widgets.MultiSelect(
        name="Sequencers",
        size=10,
        options=sequencer_names_list,
        value=sequencer_names_list,
    )

    # initial chart and table data
    slot_inclusion_line_chart = create_slot_inclusion_line_chart(
        filtered_data_dict["slot_inclusion_df"], sequencer_names_list
    )

    priority_fee_chart = create_priority_fee_chart(
        # add in filter to remove outliers and make chart look better
        filtered_data_dict["slot_inclusion_df"].filter(
            pl.col('slot_inclusion_rate') < 50),
        filtered_data_dict["slot_gas_groupby_df"].filter(
            pl.col('slot_inclusion_rate') < 50),
        sequencer_names_list,
    )

    # 6/9/24 DONT USE FOR NOW. I THINK I WILL REPLACE WITH DIFFERNT CHART?
    # avg_slot_inclusion_scatterplot = (filtered_data_dict['slot_inclusion_df'].group_by(
    #     'sequencer_names').agg(
    #     pl.col('fill_percentage').mean().alias('avg_fill_percentage'),
    #     pl.col('submission_count').mean().alias(
    #         'avg_submission_count'),
    #     pl.col('slot_inclusion_rate').mean().round(
    #         3).alias('avg_slot_inclusion_rate'),
    #     pl.col('blob_hashes_length').mean().alias(
    #         'avg_blob_hashes_length'),
    #     pl.len().alias('blob_tx_count'),
    # ).sort(by='avg_slot_inclusion_rate')
    #     .plot.scatter(x='avg_slot_inclusion_rate', y='avg_blob_hashes_length', by='sequencer_names', width=800,
    #                   height=375, xlabel='Avg Slot Inclusion Rate', ylabel='Blob Tx Size', title='Avg Slot Inclusion Rate')
    # )

    sequencer_macro_blob_table: pl.DataFrame = (
        filtered_data_dict['slot_inclusion_df'].drop_nulls().unique().group_by(
            'sequencer_names').agg(
            pl.col('fill_percentage').mean().alias('avg_fill_percentage'),
            pl.col('submission_count').mean().alias(
                'avg_submission_count'),
            pl.col('slot_inclusion_rate').mean().round(
                3).alias('avg_slot_inclusion_rate'),
            pl.col('blob_hashes_length').mean().alias(
                'avg_blob_hashes_length'),
            pl.len().alias('blob_tx_count'),
            pl.col('base_tx_fee_eth').sum().round(
                3).alias('total_base_fees_eth'),
            pl.col('priority_tx_fee_eth').sum().round(
                3).alias('total_priority_fees_eth'),
            pl.col('total_tx_fee_eth').sum().round(3).alias('total_eth_fees'),
            pl.col('priority_fee_gas').mean().round(
                3).alias('avg_priority_fee_bid'),
        ).rename({'sequencer_names': 'rollup', 'avg_blob_hashes_length': 'avg_blobs_in_tx'}))

    slot_inclusion_table_tabulator = get_slot_inclusion_table(
        filtered_data_dict["slot_inclusion_df"], sequencer_names_list)

    entire_panel = pn.Column(
        pn.Row(
            pn.pane.Markdown(
                """
            # EIP-4844 Slot Inclusion Dashboard

            ## About
            This dashboard shows detailed analytics for blob inclusion rates as well as the efficiency of using EIP-1559 priority fees
            as a bidding mechanism for faster slot inclusion. This dashboard is made using [Xatu Data](https://github.com/ethpandaops/xatu-data?tab=readme-ov-file) for EL mempool and Beacon chain data and [Hypersync](https://github.com/enviodev/hypersync-client-python) 
            for transaction gas data for the [EIP-4844 data challenge](https://esp.ethereum.foundation/data-challenge-4844).
            """
            ),
            multi_select,
            styles=dict(background="WhiteSmoke"),
        ),
        pn.pane.Markdown(
            """
            ## Historical Slot Inclusion
            When a transaction is resubmitted with updated gas parameters, the transaction hash changes. For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. 
            On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains 
            the same blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
            
            We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
            In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
            """
        ),
        pn.Row(
            slot_inclusion_line_chart.opts(axiswise=True),
            priority_fee_chart.opts(legend_position="left", show_legend=True),
            styles=dict(background="WhiteSmoke"),

        ),
        pn.Row(
            pn.pane.Markdown(
                """
            ## Slot Inclusion Rates
            **Slot Inclusion Rate** - The slot inclusion rate indicates the number of slots required for a blob to be included in the beacon chain, 
            with a higher rate signifying a longer inclusion time. The accompanying time-series chart tracks this metric from initial mempool 
            appearance to final beacon block inclusion. A 50 blob slot inclusion average is taken to smooth out the performance. 
            The target slot inclusion rate is 2. 
                """
            ),
            pn.pane.Markdown(
                """
            ## EIP-1559 Priority Fee Premium Correlation with Slot Rates
            The scatterplot illustrates the relationship between the EIP-1559 priority fee bid premiums and slot inclusion rates. The scatterplot points
            are individual blob bid datapoints and the line is a median bid premium. A higher priority fee bid premium tends to coincide 
            with longer slot inclusion times. This unexpected twist underscores the value of efficient slot utilization. The data indicates a trend 
            where higher bid premiums are associated with longer slot inclusion times, suggesting that as the time for a blob to be included 
            in the beacon chain increases, so does the priority fee bid premium. This behavior comes from the fact that if a blob sits in the 
            mempool for too long, then it is resubmitted with a higher priority fee. 
            """
            ),
            styles=dict(background="WhiteSmoke")
        ),
        pn.Row(
            pn.pane.Markdown(
                """
                ## Blob Transaction Data:
                
                This table shows various information related to the traditional transaction hashes that carry blob hashes. There is information such as
                - blob fill percentage, transaction resubmission count, number of blobs in a transaction, eth priority fees, and more.
                """
            ),
            styles=dict(background="WhiteSmoke"),
        ),
        pn.Row(
            pn.widgets.Tabulator(
                sequencer_macro_blob_table.to_pandas(), layout='fit_data'
            ),
            # avg_slot_inclusion_scatterplot,
            styles=dict(background="WhiteSmoke")
        ),
        pn.Column(
            pn.pane.Markdown(
                """
                # Slot Inclusion Data Table
                The table shows raw data for blobs that were included in the beacon chain.
                """
            ),
            slot_inclusion_table_tabulator,
            styles=dict(background="WhiteSmoke")
        )
    )

    def update_bar_chart(event):
        """
        Use this to update charts based on sequencer name user selection
        """
        entire_panel[2][0].object = create_slot_inclusion_line_chart(
            filtered_data_dict["slot_inclusion_df"],
            sequencers=multi_select.value,
        )

        entire_panel[2][1].object = create_priority_fee_chart(
            filtered_data_dict["slot_inclusion_df"],
            filtered_data_dict["slot_gas_groupby_df"],
            sequencers=multi_select.value,
        )

    multi_select.param.watch(update_bar_chart, "value")

    return entire_panel

### What charts to add in?

In [18]:
# # retrieve data from database and store in memory
cached_data = get_data()

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

2024-06-09 is within a day of 2024-06-09


In [19]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'taiko').sort(
    by='priority_tx_fee_eth', descending=True).drop_nulls()

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64
9230845,2024-06-05 13:29:23,"""0x829e82526f2ce8dc3f8294872cdcbd89ede7a7eb528fae59a4a4a900714f6069""",97.47000000000001,1,5,"""taiko""","""mainnet""",20025932,0.01472578581183566,0.00682122,43.176398978,20,0.02154700581183566,0.317
9235223,2024-06-06 04:04:59,"""0x73b13836480caf45033bd15ecee228252cdc73d0c7b68771ffdf0f9f32a1fcf6""",47.12,1,5,"""taiko""","""mainnet""",20030287,0.003258922736623892,0.006651740000000001,9.798707516,20,0.009910662736623893,0.671
9231274,2024-06-05 14:55:11,"""0x43095739f6a219e21b4161cebbd232f44c0d06ebc88d8f73d8bda8d212b2fce1""",97.59999999999998,1,2,"""taiko""","""mainnet""",20026358,0.012770053525080606,0.006299310000000001,20.272146513000003,10,0.019069363525080606,0.33
9231279,2024-06-05 14:56:11,"""0x35e3a16839690b9b5272f837ac09297221f02761991d531bf343ce690b4f8978""",97.34999999999998,1,1,"""taiko""","""mainnet""",20026363,0.012690035943260947,0.006299310000000001,20.145120566000003,10,0.018989345943260946,0.332
9235810,2024-06-06 06:02:23,"""0x17863f6c92060f8fe449b6fc847b01b8610a00545aad4e3e12e0ab7929450b74""",14.36,1,1,"""taiko""","""mainnet""",20030869,0.006113324652435006,0.006299310000000001,9.704752826,10,0.012412634652435008,0.507
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9246432,2024-06-07 17:26:47,"""0x04c7d72ea6a0a6c412401a24e00bfb78c4aaddf024ebde9f10228bfbeb4f6c6e""",9.45,1,1,"""taiko""","""mainnet""",20041434,0.004585998449697245,0.000026477900000000003,17.320098836,0.1,0.004612476349697245,0.006
9245928,2024-06-07 15:45:59,"""0xe31adb0574f49f32eb04b8f70215edead417fb5a759dab660bdb0b647e2dd32f""",6.18,1,3,"""taiko""","""mainnet""",20040932,0.006262900692297676,0.000026476700000000004,23.654385525000002,0.1,0.006289377392297676,0.004
9246023,2024-06-07 16:04:59,"""0x997e41bfb60706f19339bb54af16154c5a2062d3e20d88a49da326d1ba583580""",1.8500000000000003,1,1,"""taiko""","""mainnet""",20041027,0.005563209533086825,0.000026476700000000004,21.011717975,0.1,0.005589686233086825,0.005


In [20]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'taiko').unique().with_columns(
    pl.col('total_tx_fee_eth').sum().alias('sum'),
    pl.col('total_tx_fee_eth').mean().alias('mean')
)

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium,sum,mean
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64,f64,f64
9212352,2024-06-02 23:50:47,"""0x5584b15dcc856d4ebe3e783b924ecfd10f651f93fcc001af8ae6b9ee2789833e""",0.38000000000000006,1,2,"""taiko""","""mainnet""",20007530,0.002025642608546162,0.0018169300000000002,5.5743551170000005,5,0.003842572608546162,0.473,125.16570222590964,0.005265922092890304
9225588,2024-06-04 19:57:59,"""0xb74c31cb93abaa43b53f928842e001b2a4afaf5f85a3689a2b052ed1fa8edacc""",9.09,1,2,"""taiko""","""mainnet""",20020694,0.003616287517906816,0.0018372800000000002,9.841416436000001,5,0.005453567517906816,0.337,125.16570222590964,0.005265922092890304
9223171,2024-06-04 11:54:35,"""0xe5516aaf0091087785eab08a0c93a48048984119fd20e2f0670110995f2f388c""",1.49,1,2,"""taiko""","""mainnet""",20018289,0.002415030242133554,0.001457605,8.284241074,5,0.003872635242133554,0.376,125.16570222590964,0.005265922092890304
9243720,2024-06-07 08:24:23,"""0x4bdb63af695988e2cb1cdb166d6f7f7f9f9a3990fe1dcd776bc4aa83f5510566""",3.22,1,1,"""taiko""","""mainnet""",20038737,0.005171602704378271,0.000605002,17.096150771,2,0.005776604704378271,0.105,125.16570222590964,0.005265922092890304
9210983,2024-06-02 19:16:59,"""0x6429301aa62900cbd786b37b3f40375c54ca82d89748bfa4d0dd03ac616fb1fd""",5.96,1,1,"""taiko""","""mainnet""",20006170,0.002724743657437256,0.00164258,8.294097266,5,0.004367323657437256,0.376,125.16570222590964,0.005265922092890304
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9230260,2024-06-05 11:32:23,"""0x410b973251b0bd20eb2f4928b18261a8b40475555a4df18f153a0956294fd28f""",10.59,1,2,"""taiko""","""mainnet""",20025349,0.003989590197342498,0.001457605,13.685429857,5,0.0054471951973424975,0.268,125.16570222590964,0.005265922092890304
9240571,2024-06-06 21:54:35,"""0xbcffcdae318ad639ad4975eb04683bd5a8e3972418e4736a5d9419c6dbd614a6""",7.25,1,2,"""taiko""","""mainnet""",20035601,0.004444298227107009,0.00139631,15.914439584,5,0.005840608227107009,0.239,125.16570222590964,0.005265922092890304
9213963,2024-06-03 05:12:59,"""0x23c00a8cafb3369dcf936823e0f4573df9d560f8275be02a025f88d291e84f49""",1.54,1,1,"""taiko""","""mainnet""",20009131,0.002066973095140554,0.0014373300000000001,7.190321969,5,0.003504303095140554,0.41,125.16570222590964,0.005265922092890304


In [21]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'base').unique().with_columns(
    pl.col('total_tx_fee_eth').sum().alias('sum'),
    pl.col('total_tx_fee_eth').mean().alias('mean')
)

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium,sum,mean
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64,f64,f64
9244627,2024-06-07 11:25:47,"""0x3e18f221a8dc9a585caac3715994d7cbb40f89aceb8007d2aa496261abbef818""",98.95,1,3,"""base""","""mainnet""",20039643,0.000186157853469,0.000042000000000000004,8.864659689,2,0.000228157853469,0.184,2.092847917976109,0.0003547801183210898
9252636,2024-06-08 14:07:35,"""0x4fe26d1047e26166f331d21c31f6436df45e72ca0d04c9c4174862d7f690dcdc""",98.91000000000001,1,5,"""base""","""mainnet""",20047605,0.000162071049105,0.000042000000000000004,7.717669005,2,0.00020407104910500002,0.206,2.092847917976109,0.0003547801183210898
9257616,2024-06-09 06:43:35,"""0xf2458add44bd03abb5ae55495f08141c3a07124f7745a4a20ab427bece6b03c3""",99.32,1,2,"""base""","""mainnet""",20052558,0.000087146268237,0.000042000000000000004,4.149822297,2,0.000129146268237,0.325,2.092847917976109,0.0003547801183210898
9245901,2024-06-07 15:40:35,"""0x3b8fea724c4b9f1a09b511d48078ab534d2cb651cb279482935ef9d6fb98ecfb""",99.7,1,3,"""base""","""mainnet""",20040905,0.00043930363415400003,0.000042000000000000004,20.919220674,2,0.000481303634154,0.087,2.092847917976109,0.0003547801183210898
9215902,2024-06-03 11:40:47,"""0xe96b64c68213ffec0630a019a5740780328c475ba0eac93bc0f42c1fcfe7a3de""",98.8,1,17,"""base""","""mainnet""",20011058,0.00018410076516300002,0.00008400000000000001,8.766703103000001,4,0.00026810076516300003,0.313,2.092847917976109,0.0003547801183210898
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9237240,2024-06-06 10:48:23,"""0xc2f29729bf3f856a22671c139460d646a86b4c7453d655425c913bd49df4c194""",99.33999999999999,1,2,"""base""","""mainnet""",20032291,0.00039305153244,0.000042000000000000004,18.71673964,2,0.00043505153244,0.097,2.092847917976109,0.0003547801183210898
9240625,2024-06-06 22:05:23,"""0x5e26e8558b157d23e767089bf6e512a5a322deca5ad024e0eac620eaac9d660e""",97.96999999999998,1,1,"""base""","""mainnet""",20035655,0.00029239700406,0.000042000000000000004,13.92366686,2,0.00033439700406,0.126,2.092847917976109,0.0003547801183210898
9245212,2024-06-07 13:22:47,"""0xbd15b7b004b76f179b8c58b2fc72ce2a95b5aa6efdbe9ea8d24331829edeea2a""",98.3,1,1,"""base""","""mainnet""",20040223,0.00034522640193300003,0.000042000000000000004,16.439352473,2,0.000387226401933,0.108,2.092847917976109,0.0003547801183210898


In [22]:
# VERIFY TAIKO FEE USAGE

In [23]:
slot_inclusion_joined_df.drop_nulls().unique().group_by('sequencer_names').agg(
    pl.col('base_tx_fee_eth').sum().alias('total_base_fees_eth'),
    pl.col('priority_tx_fee_eth').sum().alias('total_priority_fees_eth'),
    pl.col('total_tx_fee_eth').sum().alias('total_eth_fees'),
    pl.col('priority_fee_gas').mean().alias('avg_priority_gas_bid'),
).sort(by='sequencer_names')

sequencer_names,total_base_fees_eth,total_priority_fees_eth,total_eth_fees,avg_priority_gas_bid
str,f64,f64,f64,f64
"""arbitrum""",11.27932051910018,0.9040326159999994,12.183353135100182,1.0045558086560364
"""base""",1.6155599179761089,0.47728800000000227,2.092847917976109,3.8528564163417527
"""blast""",0.49569862846297497,0.14697900000000017,0.642677628462975,3.7487948580610606
"""linea""",5.56231571164354,0.611375529346341,6.173691240989879,1.3913711816075263
"""metal""",0.15280653572732097,0.033641999999999984,0.186448535727321,2.005006257822278
"""optimism""",0.763579147921065,0.1388100000000005,0.9023891479210652,2.288781163434903
"""paradex""",3.375864514535886,0.02809896480000004,3.4039634793358857,0.09999999999999978
"""scroll""",13.294968835797041,0.10918766425740803,13.404156500054453,0.09704550746164445
"""taiko""",88.73702521420964,36.428677011699854,125.16570222590963,4.981337035634651


### run the dashboard

In [24]:
# # retrieve data from database and store in memory
cached_data = get_data()

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'blob_hashes_length', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

filtered_data_dict = filter_data_seq(
    sequencers_l2["sequencer_names"], slot_inclusion_joined_df, cached_data
)

sequencer_names_list: list[str] = sorted(sequencers_l2["sequencer_names"])

dashboard = start_interactive_panel(filtered_data_dict, sequencer_names_list)
dashboard.servable()

2024-06-09 is within a day of 2024-06-09
Launching server at http://localhost:43567


Gtk-Message: 15:41:36.438: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:41:36.439: Failed to load module "canberra-gtk-module"


Opening in existing browser session.
